# 第六章 图像变换     

## 卷积     

我们在第 5 章看到的所有操作都可以被理解成普通卷积的特殊情况.    

一个特殊卷积所实现的功能是由其卷积核的形式决定的.这个核本质上是一个大小固定,由数值参数构成的数组, 数组的参考点通常位于数组的中心.   

对于每一个 kernel, 我们可以得到图像中某个点与 anchor 重合的像素值和 kernel 覆盖的其余邻近值, 将这些值分别和 kernel 对应位置上的值相乘并求和, 并将这个结果放在图像中与 anchor 重合的位置.    

**cv2.filter2D() 函数**:    
- cv2.filter2D(src, ddepth, kernel[, dst[, anchor[, delta[, borderType]]]]) → dst;    
- ddepth - 指定输出图像的 depth; 如果为负数(ddepth=-1), 输出图像的 depth 就和 src.depth() 相同; 以下是支持的原图 depth 和输出图 depth:    
```
src.depth() = CV_8U, ddepth = -1/CV_16S/CV_32F/CV_64F
src.depth() = CV_16U/CV_16S, ddepth = -1/CV_32F/CV_64F
src.depth() = CV_32F, ddepth = -1/CV_32F/CV_64F
src.depth() = CV_64F, ddepth = -1/CV_64F
```
- kernel – 卷积核(或者是 correlation kernel).      
- anchor – 可选参数, 卷积核的参考点, 表示被滤波的点在 kernel 中的相对位置. 默认值是 (-1,-1) , 表示 anchor 在 kernel 的中心.    
- delta – 可选参数, 将最后计算出的值保存在输出图像之前会加上这个值.    
- borderType – 可选参数, 边界像素的外推方法(常用的有 zero padding 和复制外推法).     

**Note**:   

1. 卷积核用一个单通道的浮点数矩阵表示; 如果你想在不同的通道上引用不同的卷积核, 那么应该使用 split() 函数将原图且分到独立的图像平面上, 然后单独执行卷积运算.    

2. 该函数执行的是线性滤波, 支持 in-place 操作.    

3. 对于小的 kernel, 使用直接法进行计算; 对于大的 kernel(比如 ``11 x 11``), 使用基于 DFT 的算法加速.    



In [1]:
## 卷积操作, 尝试着换换别的 kernel
import cv2
import numpy as np

image = cv2.imread("../pictures_for_code/building.jpg", cv2.IMREAD_GRAYSCALE)
kernel = np.array([[-3.0, 0, 3], [-10, 0, 10], [-3, 0, 3]], dtype = np.float)  # Scharr 滤波器
conv = cv2.filter2D(image, -1, kernel, borderType = cv2.BORDER_REPLICATE)
print("shape 1 2: %s, %s" % (image.shape,  conv.shape))

cv2.imshow("image", image)
cv2.imshow("conv", conv)

key = cv2.waitKey(0)
if key:
    cv2.destroyAllWindows()

shape 1 2: (360, 480), (360, 480)


### 卷积边界     

当卷积点在图像边界会发生什么? 使用 cv2.copyMakeBorder() 可以在图像四周生成一个边界, 有效地解决这个问题.    

**cv2.copyMakeBorder() 函数**:     
- cv2.copyMakeBorder(src, top, bottom, left, right, borderType[, dst[, value]]) → dst;    
- top/bottom/left/right: 指定四个方向需要外推的像素个数. 例如, top=1, bottom=1, left=1, right=1 就是创建 1 个像素宽的边界.   
- borderType: 创建边界的方式. 常用的是 BORDER_REPLICATE 和 BORDER_CONSTANT.   
- value: 当 borderType==BORDER_CONSTANT 时需要指定的边界值.   

**Note**:    
当源图像是新图像的一部分(ROI)时, 这个函数会直接使用 ROI 外部的像素生成边界. 如果想要禁用这个功能,即并不把源图像当成是一个 ROI, 时刻使用外推法生成新的边界, 使用 borderType | BORDER_ISOLATED.    


In [2]:
## 卷积边界填充
import cv2

border = 5

image = cv2.imread("../pictures_for_code/building.jpg", cv2.IMREAD_GRAYSCALE)
border_replicate = cv2.copyMakeBorder(image, border, border, border, border, borderType = cv2.BORDER_REPLICATE)
border_constant = cv2.copyMakeBorder(image, border, border, border, border, borderType = cv2.BORDER_CONSTANT, value = 0)
print("shape 1 2: %s, %s" % (border_constant.shape,  border_replicate.shape))

cv2.imshow("border_replicate", border_replicate)
cv2.imshow("border_constant", border_constant)

key = cv2.waitKey(0)
if key:
    cv2.destroyAllWindows()

shape 1 2: (370, 490), (370, 490)


## 梯度和 Sobel 导数    

一个最重要且最基本的卷积操作就是导数的计算(其实是近似值).   

通常来说, 用来表达微分的最常用的操作是 Sobel 微分算子. Sobel 算子包含任意阶的微分以及融合偏导.   

**cv2.Sobel()**:    
- 使用扩展的 Sobel 算子计算图像的一阶, 二阶, 三阶或者复合导数.    
- cv2.Sobel(src, ddepth, xorder, yorder[, dst[, ksize[, scale[, delta[, borderType]]]]]) → dst;    
- ddepth - 指定输出图像的 depth; 如果为负数(ddepth=-1), 输出图像的 depth 就和 src.depth() 相同; 以下是支持的原图 depth 和输出图 depth:    
```
src.depth() = CV_8U, ddepth = -1/CV_16S/CV_32F/CV_64F
src.depth() = CV_16U/CV_16S, ddepth = -1/CV_32F/CV_64F
src.depth() = CV_32F, ddepth = -1/CV_32F/CV_64F
src.depth() = CV_64F, ddepth = -1/CV_64F
```
如果是 8-bit 的输入图片, 返回的是裁切后的 derivatives.
- xorder – x 方向的导数的阶数.    
- yorder – y 方向的导数的阶数.     
- ksize – 扩展 Sobel kernel 的 size, 可选值为 1, 3, 5, or 7.   
- scale – 可选参数, 计算导数时的缩放因子, 默认值为 1, 即不缩放.   
- delta – 可选参数, 再将结果保存到 dst 之前加上这个值.   
- borderType – 边界像素的外推方法.   

Sobel 算子结合了高斯平滑和微分运算, 因此结果会或多或少的抗噪声干扰. 最常见的调用方式是以下两种:    
- ( xorder = 1, yorder = 0, ksize = 3)  计算 x 的一阶导数.    
- ( xorder = 0, yorder = 1, ksize = 3) 计算 y 的一阶导数.    
- 第一种情况对应的 kernel 为: `[[-1.0, 0, 1], [-2, 0, 2], [-1, 0, 1]]`.    
- 第二种情况对应的 kernel 为: `[[-1.0, -2, -1], [0, 0, 0], [1, 2, 1]]`.    
 
Sobel 算子有一个很好的性质, 即可以定义任意大小的核. 并且这些核可以快速用迭代方式构造. 大核对导数有更好的逼近, 因为小核对噪声更敏感.    

Sobel 导数并不是真正的导数, 因为 Sobel 算子定义在离散空间上,. Sobel 算子真正表示的是多项式拟合, 也就是说 x 方向上的二阶 Sobel 导数并不是真正的二阶导数, 而是对抛物线函数的局部拟合. 使用较大核可以在更多像素上计算这种拟合.      

In [3]:
## Sobel 算子的两种常用 kernel (也可以用卷积直接实现)
import cv2
import numpy as np

image = cv2.imread("../pictures_for_code/building.jpg", cv2.IMREAD_GRAYSCALE)
kernel_x = np.array([[-1.0, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype = np.float)    # Sobel 算子
kernel_y = np.array([[-1.0, -2, -1],[0, 0, 0],[1, 2, 1]], dtype = np.float)    # Sobel 算子

conv_x = cv2.filter2D(image, -1, kernel_x, borderType = cv2.BORDER_REPLICATE)
conv_y = cv2.filter2D(image, -1, kernel_y, borderType = cv2.BORDER_REPLICATE)

sobel_x = cv2.Sobel(image, -1, 1, 0, 3)
sobel_y = cv2.Sobel(image, -1, 0, 1, 3)

cv2.imshow("origin", image)
cv2.imshow("conv_x", conv_x)
cv2.imshow("conv_y", conv_y)
cv2.imshow("sobel_x", sobel_x)
cv2.imshow("sobel_y", sobel_y)

key = cv2.waitKey(0)
if key:
    cv2.destroyAllWindows()

### Scharr 滤波器     

对于小的核, Sobel 算子的计算精度会大大降低. 这时可私用 Scharr 滤波器解决 Sobel 算子计算的精度问题, 二者速度是一样快的.   

对于 cv2.Sobel() 函数中的 ksize 参数值, 当指定 ksize = CV_SCHARR (即 ksize = -1)时, 使用的是 3x3 的 Scharr 滤波器.    

Scharr 使用的核是: `[[-3.0, 0, 3], [-10, 0, 10], [-3, 0, 3]]`.    

**cv2.Scharr() 函数**:    
- cv2.Scharr(src, ddepth, dx, dy[, dst[, scale[, delta[, borderType]]]]) → dst;    


In [4]:
## Scharr 滤波器 
import cv2
import numpy as np

image = cv2.imread("../pictures_for_code/building.jpg", cv2.IMREAD_GRAYSCALE)

sobel_x = cv2.Sobel(image, -1, 1, 0, ksize = -1)    # 等价于 cv2.Scharr()   
scharr = cv2.Scharr(image, -1, 1, 0)

cv2.imshow("sobel_x", sobel_x)
cv2.imshow("scharr", scharr)

key = cv2.waitKey(0)
if key:
    cv2.destroyAllWindows()

### 拉普拉斯变换     

拉普拉斯算子是计算沿着 X 轴和 Y 轴的二次导数的和, 这就意味着周围是更高值的单点会使函数值最大化, 反过来, 周围是更低值的点将会使函数的负值最大化.   

基于这种思想, 可以使用拉普拉斯算子做边缘检测.当我们逼近类似边缘的不连续区域时导数会快速增长,而穿过这些不连续地方时导数又会快速减小.所以导数会在此范围内有局部极值.这样局部最大值有可能位于二阶导数为 0 的地方.因此我们可以通过一阶导数和二阶导数均为 0 来过滤那些不是极值点的检测.     

拉普拉斯变换使用 Sobel 算子计算 x 和 y 的二阶导数之和.    

**cv2.Laplacian() 函数**:    
- cv2.Laplacian(src, ddepth[, dst[, ksize[, scale[, delta[, borderType]]]]]) → dst;    
- ksize – kernel size,正数且为奇数. 用于计算二阶导数的滤波器.    
- scale – 可选参数, 计算导数时的缩放因子, 默认值为 1, 即不缩放.   
- delta – 可选参数, 再将结果保存到 dst 之前加上这个值.   
- borderType – 边界像素的外推方法.

当 ksize > 1 时使用 Sobel 算子计算 x 和 y 的二阶导数之和.     

当 ksize == 1 时, 使用下面的 kernel 计算 Laplacian 变换: ``[[0, 1, 0], [1, -4, 1], [0, 1, 0]]``.    

In [5]:
import cv2

img = cv2.imread("../pictures_for_code/building.jpg", 0)

laplacian = cv2.Laplacian(img,cv2.CV_64F)
sobelx = cv2.Sobel(img,cv2.CV_64F,1,0,ksize=5)
sobely = cv2.Sobel(img,cv2.CV_64F,0,1,ksize=5)

cv2.imshow("sobelx", sobelx)
cv2.imshow("sobely", sobely)
cv2.imshow("laplacian", laplacian)

key = cv2.waitKey(0)
if key:
    cv2.destroyAllWindows()

## Canny 算子    

以上拉普拉斯边缘检测描述的边缘检测方法在 1986 年由 J.Canny 得到完善, 也就是通常所说的 Canny 边缘检测法.   

不同于拉普拉斯边缘检测算法的是:首先在 x 和 y 方向求一阶导数, 然后组合为 4 个方向的导数. 这些方向导数达到局部最大值的点就是组成边缘的候选点.   

Canny 算法最重要的一个新特点是试图将独立边的候选像素拼接成轮廓. 轮廓的形成是对这些像素运用滞后性阈值.这意味着有两个阈值,上限和下限. 如果一个像素的梯度大于上限阈值, 则被认为是边缘像素, 如果低于下限阈值, 则被抛弃. 如果介于二者之间, 只有当其与高于上限阈值的像素连接时才会被接受.   

Canny 推荐的上下限阈值比为 2:1 到 3:1 之间. 小的阈值太小的话就会得到很密集的边缘, 因为大的阈值是用来确定初始化的强边缘的.      

**cv2.Canny() 函数**:    
- cv2.Canny(image, threshold1, threshold2[, edges[, apertureSize[, L2gradient]]]) → edges;    
- image – 8-bit input image.  
- edges – output edge map; single channels 8-bit image, which has the same size as image .  
- threshold1 – first threshold for the hysteresis procedure.  
- threshold2 – second threshold for the hysteresis procedure.  
- apertureSize – aperture size for the Sobel() operator.  
- L2gradient – a flag, indicating whether a more accurate  L_2 norm  =\sqrt{(dI/dx)^2 + (dI/dy)^2} should be used to calculate the image gradient magnitude ( L2gradient=true ), or whether the default  L_1 norm =|dI/dx|+|dI/dy| is enough ( L2gradient=false ).  

小的阈值用来连接边缘, 大的阈值用来确定初始化的强边缘.See http://en.wikipedia.org/wiki/Canny_edge_detector.   

**Note**:   
canny edge detector 的实例代码可以在 samples/cpp/edge.cpp,或 samples/python/edge.py 中找到.    

In [6]:
## Canny 上下限不同比例的比较(3:2, 5:1)    

img = cv2.imread("../pictures_for_code/building.jpg", 0)

canny_32 = cv2.Canny(img,100, 150)
canny_51 = cv2.Canny(img,10, 50)

cv2.imshow("canny_32", canny_32)
cv2.imshow("canny_51", canny_51)

key = cv2.waitKey(0)
if key:
    cv2.destroyAllWindows()

## 霍夫变换    

霍夫变换是一种在图像中寻找直线, 圆以及其他简单集合形状的方法.   

原始的霍夫变换是一种直线变换, 即在二值图像中寻找直线的一种相对快速的方法.变换可以推广到其他普通的情况, 而不仅仅是直线.   

### 霍夫线变换     

霍夫线变换的基本理论是二值图像中任何点都可能是一些候选直线集合的一部分.   

OpenCV 支持两种不同形式的霍夫变换: 标准霍夫变换(SHT) 和累计概率霍夫变换(PPHT).    

**cv2.HoughLines() 和 cv2.HoughLinesP()**:    
- 使用霍夫线变换在二值图中计算直线.
- cv2.HoughLines(image, rho, theta, threshold[, lines[, srn[, stn[, min_theta[, max_theta]]]]]) → lines;    
- cv2.HoughLinesP(image, rho, theta, threshold[, lines[, minLineLength[, maxLineGap]]]) → lines;    
- rho – 累加器的距离分辨率, 单位是像素.    
- theta – 累加器的角度分辨率, 单位是弧度.     
- threshold – 累加器的阈值. 只有投票值大于阈值的那些直线才会被接受.   
- lines – Output vector of lines. Each line is represented by a two-element vector  (\rho, \theta) .  \rho is the distance from the coordinate origin  (0,0) (top-left corner of the image).  \theta is the line rotation angle in radians ( 0 \sim \textrm{vertical line}, \pi/2 \sim \textrm{horizontal line} ).
- srn – For the multi-scale Hough transform, it is a divisor for the distance resolution rho . The coarse accumulator distance resolution is rho and the accurate accumulator resolution is rho/srn . If both srn=0 and stn=0 , the classical Hough transform is used. Otherwise, both these parameters should be positive.
- stn – For the multi-scale Hough transform, it is a divisor for the distance resolution theta.
- min_theta – For standard and multi-scale Hough transform, minimum angle to check for lines. Must fall between 0 and max_theta.
- max_theta – For standard and multi-scale Hough transform, maximum angle to check for lines. Must fall between min_theta and CV_PI.    
- minLineLength – Minimum line length. Line segments shorter than that are rejected.   
- maxLineGap – Maximum allowed gap between points on the same line to link them.    
- method –
```
One of the following Hough transform variants:

CV_HOUGH_STANDARD 标准霍夫变换. Every line is represented by two floating-point numbers  (\rho, \theta) , where  \rho is a distance between (0,0) point and the line, and  \theta is the angle between x-axis and the normal to the line. Thus, the matrix must be (the created sequence will be) of CV_32FC2 type
CV_HOUGH_PROBABILISTIC 概率霍夫变换 (more efficient in case if the picture contains a few long linear segments). It returns line segments rather than the whole line. Each segment is represented by starting and ending points, and the matrix must be (the created sequence will be) of the CV_32SC4 type.
CV_HOUGH_MULTI_SCALE 多尺度变量霍夫变换. The lines are encoded the same way as CV_HOUGH_STANDARD.
param1 –
First method-dependent parameter:

For the classical Hough transform, it is not used (0).
For the probabilistic Hough transform, it is the minimum line length.
For the multi-scale Hough transform, it is srn.
param2 –
Second method-dependent parameter:

For the classical Hough transform, it is not used (0).
For the probabilistic Hough transform, it is the maximum gap between line segments lying on the same line to treat them as a single line segment (that is, to join them).
For the multi-scale Hough transform, it is stn.
```

The function implements the probabilistic Hough transform algorithm for line detection, described in [Matas00]. See the line detection example below:

/* This is a standalone program. Pass an image name as the first parameter
of the program.  Switch between standard and probabilistic Hough transform
by changing "#if 1" to "#if 0" and back */

The function implements the standard or standard multi-scale Hough transform algorithm for line detection. See http://homepages.inf.ed.ac.uk/rbf/HIPR2/hough.htm for a good explanation of Hough transform. See also the example in HoughLinesP() description.


In [33]:
## 概率霍夫变换和标准霍夫变换
import cv2
import numpy as np

image = cv2.imread("../pictures_for_code/building.jpg", 0)
canny = cv2.Canny(image, 50, 200, 3 )
result = cv2.cvtColor(canny, cv2.COLOR_GRAY2BGR)
result_p = result.copy()    

# 概率霍夫变换
lines_p = cv2.HoughLinesP(canny.copy(), 1, np.pi/180, 80, 30, 10)
for line in lines_p:
    line = line[0]
    cv2.line( result_p, (line[0], line[1]),(line[2], line[3]), (0,255,0), 2, 8 )

# 标准霍夫变换
lines_s = cv2.HoughLines(canny.copy(), 1, np.pi/180, 100 )   #这里对最后一个参数使用了经验型的值   
result_s = result.copy()    
for line in lines_s:   
    line = line[0]
    
    rho = line[0]  #第一个元素是距离 rho    
    theta= line[1] #第二个元素是角度 theta    
    #print(rho, theta)    
    if  (theta < (np.pi/4. )) or (theta > (3.*np.pi/4.0)): #垂直直线    
        #该直线与第一行的交点    
        pt1 = (int(rho/np.cos(theta)),0)    
        #该直线与最后一行的交点    
        pt2 = (int((rho-result.shape[0]*np.sin(theta))/np.cos(theta)),result.shape[0])    
        cv2.line( result_s, pt1, pt2, (255))      #绘制一条白线 
    else: #水平直线    
        # 该直线与第一列的交点    
        pt1 = (0,int(rho/np.sin(theta)))    
        #该直线与最后一列的交点    
        pt2 = (result.shape[1], int((rho-result.shape[1]*np.cos(theta))/np.sin(theta)))    
        cv2.line(result_s, pt1, pt2, (255), 1)    #绘制一条直线  

cv2.imshow("Source", image)
cv2.imshow("HoughLine_P", result_p)
cv2.imshow("HoughLine", result_s)

key = cv2.waitKey(0)
if key:
    cv2.destroyAllWindows()

### 霍夫圆变换      

霍夫圆变换和霍夫直线变化大体上是类似的.因为圆的参数化需要三维参数(x, y, r), 这会需要大量的内存且运算速度很慢.       

OpenCV 中通过一个比较灵活的霍夫梯度法来解决圆变换这一问题.    

**霍夫圆变换的一些缺点**:    
- Sobel 算子计算局部梯度的精度问题可能会引入噪声;   
- 在边缘图像中的整个非 0 像素集被认为是每个中心的候选. 因此, 如果把累加器阈值设置的偏低, 算法就会消耗很长时间;    
- 每个中心只选择一个圆, 如果有同心圆, 则只能选择一个;    

**cv2.HoughCircles() 函数**:    
- 使用霍夫变换查找灰度图(而不是用二值图)中的圆;   
- cv2.HoughCircles(image, method, dp, minDist[, circles[, param1[, param2[, minRadius[, maxRadius]]]]]) → circles;   
- circles – Output vector of found circles. Each vector is encoded as a 3-element floating-point vector (x, y, radius) .
circle_storage – In C function this is a memory storage that will contain the output sequence of found circles.
- method – Detection method to use. Currently, the only implemented method is CV_HOUGH_GRADIENT , which is basically 21HT , described in [Yuen90].
- dp – Inverse ratio of the accumulator resolution to the image resolution. For example, if dp=1 , the accumulator has the same resolution as the input image. If dp=2 , the accumulator has half as big width and height.
- minDist – Minimum distance between the centers of the detected circles. If the parameter is too small, multiple neighbor circles may be falsely detected in addition to a true one. If it is too large, some circles may be missed.
- param1 – First method-specific parameter. In case of CV_HOUGH_GRADIENT , it is the higher threshold of the two passed to the Canny() edge detector (the lower one is twice smaller).
- param2 – Second method-specific parameter. In case of CV_HOUGH_GRADIENT , it is the accumulator threshold for the circle centers at the detection stage. The smaller it is, the more false circles may be detected. Circles, corresponding to the larger accumulator values, will be returned first.
- minRadius – Minimum circle radius.
- maxRadius – Maximum circle radius.

**Note** Usually the function detects the centers of circles well. However, it may fail to find correct radii. 
You can assist to the function by specifying the radius range ( minRadius and maxRadius ) if you know it.
Or, you may ignore the returned radius, use only the center, and find the correct radius using an additional
procedure.

In [ ]:
#include <opencv2/imgproc.hpp>
#include <opencv2/highgui.hpp>
#include <math.h>

using namespace cv;

int main(int argc, char** argv)
{
    Mat img, gray;
    if( argc != 2 && !(img=imread(argv[1], 1)).data)
        return -1;
    cvtColor(img, gray, COLOR_BGR2GRAY);
    // smooth it, otherwise a lot of false circles may be detected
    GaussianBlur( gray, gray, Size(9, 9), 2, 2 );
    vector<Vec3f> circles;
    HoughCircles(gray, circles, HOUGH_GRADIENT,
                 2, gray->rows/4, 200, 100 );
    for( size_t i = 0; i < circles.size(); i++ )
    {
         Point center(cvRound(circles[i][0]), cvRound(circles[i][1]));
         int radius = cvRound(circles[i][2]);
         // draw the circle center
         circle( img, center, 3, Scalar(0,255,0), -1, 8, 0 );
         // draw the circle outline
         circle( img, center, radius, Scalar(0,0,255), 3, 8, 0 );
    }
    namedWindow( "circles", 1 );
    imshow( "circles", img );
    return 0;
}


## 重映射    

具体来说, 许多变换都有一个共同点, 就是把一幅图像中的一个位置的像素重映射到另一个位置.  

OpenCV 提供了 cv2.remap() 函数;     

cv2.remap() 的一个通常的用途是校正标定和立体的图像.       

**cv2.remap()**:    
- cv2.remap(src, map1, map2, interpolation[, dst[, borderMode[, borderValue]]]) → dst;   
- dst: 输出图像. 其 size 和 map1 相同, 像素值数据的 type 和 src 相同.    
- map1: 第一个 map, 可以指定 x,y 点或仅仅是 x 的值, 类型可以是 CV_16SC2 , CV_32FC1 或 CV_32FC2. 将一个浮点数表示转换为定点数表示, 可以加速计算.   
- map2: 第二个 map, 指定 y 值, 类型可以是 CV_16UC1 , CV_32FC1. 如果 map1 指定的是 (x,y) 点, 则为空.   
- interpolation – 插值的方法, 除了不支持 INTER_AREA 方法外, 和 resize() 支持的插值方法相同.    
- borderMode – Pixel extrapolation method (see borderInterpolate() ). When borderMode=BORDER_TRANSPARENT , it means that the pixels in the destination image that corresponds to the “outliers” in the source image are not modified by the function.    
- borderValue – 当 borderMode==BORDER_CONSTANT 时设置像素值. 默认为 0.      

The function remap transforms the source image using the specified map:

\texttt{dst} (x,y) =  \texttt{src} (map_x(x,y),map_y(x,y))

where values of pixels with non-integer coordinates are computed using one of available interpolation methods. map_x and map_y can be encoded as separate floating-point maps in map_1 and map_2 respectively, or interleaved floating-point maps of (x,y) in map_1 , or fixed-point maps created by using convertMaps() . The reason you might want to convert from floating to fixed-point representations of a map is that they can yield much faster (~2x) remapping operations. In the converted case, map_1 contains pairs (cvFloor(x), cvFloor(y)) and map_2 contains indices in a table of interpolation coefficients.

该函数不支持 in-place 操作.

## 拉伸, 收缩, 扭曲和旋转     

可以用来人为的扩大一组训练图像, 用于物体识别.    

对于平面区域, 有两种方式的几何变换:   
- 一种是基于 2x3 矩阵的仿射变换;    
- 另一种是基于 3x3 矩阵的透视变换或单应性变换;     

### 仿射变换     

一个任意的仿射变换可以表达为乘以一个 2x2 矩阵再加上一个 2x1 向量的形式, 可以转换为 2x3 矩阵.    

仿射变换可以将矩形转换成平行四边形, 也可以将矩形旋转或者按比例变换.   

## 透视变换     

透视变换提供了更大的灵活性, 透视变换可以将矩形转变梯形. 因为平行四边形也是特殊的梯形, 因此仿射变换是透视变换的子集.    

TODO: 基础理论深入.    